## Ping pong

Two processes ping pong a token back and forth, incrementing it until it reaches a given value.

1. Process 0 sends a message to process 1 (ping).

2. After receiving this message, process 1 sends a message back to process 0 (pong).

3. Each time a message is sent, the token is incremented by 1.

4. Repeat this ping pong until the value of token reaches 6, i.e. 3 pings and 3 pongs.

***
#### C skeleton

In [ ]:
#include <mpi.h>

In [ ]:
%%executable  a.x -- -lmpi
int i, rank;
float token[1] = {12};
MPI_Status status;

MPI_Init(NULL, NULL);
MPI_Comm_rank(MPI_COMM_WORLD, &rank);

if (rank == 0)
{
  printf("I am %i before send ping \n", rank);
  MPI_Send(token, 1, MPI_FLOAT, 1, 17, MPI_COMM_WORLD);
  //add your code here
}
else if (rank == 1)
{
  MPI_Recv(token, 1, MPI_FLOAT, 0, 17, MPI_COMM_WORLD, &status);
  printf("I am %i after recv ping \n", rank);
  //add your code here
}

MPI_Finalize();

Now compile it and run it with 4 processes. 

In [ ]:
!mpirun -np 4 a.x

***
#### Python skeleton

In [ ]:
%%file ping.py
from mpi4py import MPI

comm = MPI.COMM_WORLD
rank = comm.Get_rank()
token = [12]
status = MPI.Status()

if rank == 0:
    print("I am %d before send ping" % rank)
    comm.send(token, dest=1)
    #add your code here
if rank == 1:
    token = comm.recv(source=0)
    print("I am %d after recv ping" % rank)
    #add your code here

Now compile it and run it with 4 processes. 

In [ ]:
!mpirun -np 4 python ping.py

***
#### Fortran skeleton

In [ ]:
%%file ping.f90
program ping
use mpi

integer ( kind = 4 ) error
integer ( kind = 4 ) rank
integer :: status(MPI_STATUS_SIZE)
real :: token(1)
token = 12.0

call MPI_Init(error);
call MPI_Comm_rank(MPI_COMM_WORLD, rank, error);
if (rank .eq. 0) then
    print *, 'I am process', rank, ' before send ping'
    call MPI_Send(token, 1, MPI_REAL, 1, 17, MPI_COMM_WORLD, error)
    ! add your code here
else if (rank .eq. 1) then
    call MPI_Recv(token, 1, MPI_REAL, 0, 17, MPI_COMM_WORLD, status, error)
    print *, 'I am process', rank, ' after recv ping'
    ! add your code here
end if
call MPI_Finalize(error)
end

Now compile it and run it with 4 processes. 

In [ ]:
!mpif90 ping.f90 && mpirun -np 4 a.out

***

### You can compare with our solution:

***
#### C solution

In [ ]:
%%executable  a.x -- -lmpi
int rank;
float token[1] = {12};
MPI_Status status;

MPI_Init(NULL, NULL);
MPI_Comm_rank(MPI_COMM_WORLD, &rank);

if (rank == 0)
{
  while (token[0] > 6) {
      printf("I am %i before send ping \n", rank);
      token[0]--;
      MPI_Send(token, 1, MPI_FLOAT, 1, 17, MPI_COMM_WORLD);
      MPI_Recv(token, 1, MPI_FLOAT, 1, 23, MPI_COMM_WORLD, &status);
      printf("I am %i after recv pong \n", rank);
  }
}
else if (rank == 1)
{
  while (token[0] > 6) {
      MPI_Recv(token, 1, MPI_FLOAT, 0, 17, MPI_COMM_WORLD, &status);
      printf("I am %i after recv ping \n", rank);
      printf("I am %i before send pong \n", rank);
      token[0]--;
      MPI_Send(token, 1, MPI_FLOAT, 0, 23, MPI_COMM_WORLD);
  }
}

MPI_Finalize();

In [ ]:
!mpirun -np 4 a.x

***
#### Python solution

In [ ]:
%%file ping.py
from mpi4py import MPI

comm = MPI.COMM_WORLD
rank = comm.Get_rank()
token = [12]
status = MPI.Status()

if rank == 0:
    while token[0] > 6:
        print("I am %d before send ping" % rank)
        token[0] -= 1
        comm.send(token, dest=1)
        token = comm.recv(source=1)
        print("I am %d after recv pong" % rank)
if rank == 1:
    while token[0] > 6:
        token = comm.recv(source=0)
        print("I am %d after recv ping" % rank)
        print("I am %d before send pong" % rank)
        token[0] -= 1
        comm.send(token, dest=0)

In [ ]:
!mpirun -np 4 python ping.py

***
#### Fortran solution

In [ ]:
%%file ping.f90
program ping
use mpi

integer ( kind = 4 ) error
integer ( kind = 4 ) rank
integer :: status(MPI_STATUS_SIZE)
real :: token(1)
token = 12.0

call MPI_Init(error);
call MPI_Comm_rank(MPI_COMM_WORLD, rank, error);
if (rank .eq. 0) then
    do while (token .gt. 6.0)
        print *, 'I am process', rank, ' before send ping'
        token = token - 1
        call MPI_Send(token, 1, MPI_REAL, 1, 17, MPI_COMM_WORLD, error)
        call MPI_Recv(token, 1, MPI_REAL, 1, 23, MPI_COMM_WORLD, status, error)
        print *, 'I am process', rank, ' after recv pong'
    end do
else if (rank .eq. 1) then
    do while (token .gt. 6.0)
        call MPI_Recv(token, 1, MPI_REAL, 0, 17, MPI_COMM_WORLD, status, error)
        print *, 'I am process', rank, ' after recv ping'
        print *, 'I am process', rank, ' before send pong'
        token = token - 1
        call MPI_Send(token, 1, MPI_REAL, 0, 23, MPI_COMM_WORLD, error)
    end do
end if
print *, token
call MPI_Finalize(error)
end

In [ ]:
!mpif90 ping.f90 && mpirun -np 4 a.out